In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy.modflow as mf
import flopy.utils as fu
import shutil
import json
from scipy.misc import imresize

%matplotlib inline

In [2]:
central_base_model_path = os.path.join('central-base-model')
landelino_model_path = os.path.join('landelino-model')

In [3]:
if os.path.exists(landelino_model_path):
    shutil.rmtree(landelino_model_path)
    
if not os.path.exists(landelino_model_path):
    os.makedirs(landelino_model_path)

In [4]:
extractRowsBetween = [6, 20]
extractColsBetween = [49, 60]
scaling_factor = 10

def scale(factor, array):
    if (len(array.shape) == 2):
        layer = []
        for row in array:
            newRow = np.array(list(map(lambda v: np.ones(factor) * v, row))).reshape(-1)
            for _ in range(factor): # or xrange if you are on 2.X
                layer.append(newRow)

        return layer
    
    if (len(array.shape) == 3):
        layers = []
        for layerData in array:
            layer = []
            for row in layerData:
                newRow = np.array(list(map(lambda v: np.ones(factor) * v, row))).reshape(-1)
                for _ in range(factor): # or xrange if you are on 2.X
                    layer.append(newRow)

            layers.append(layer)

        return layers

### Setup mf-package

In [5]:
modelname = 'landelina'
ml = mf.Modflow(modelname=modelname, exe_name='mf2005', version='mf2005', model_ws=landelino_model_path)

### Setup dis-package

In [6]:
nlay = 7
nrow = 140
ncol = 110

area_width_y = 1400
area_width_x = 1100

delc = area_width_x/ncol
delr = area_width_y/nrow

nper = 1

In [7]:
cbm = mf.Modflow.load("model.nam", model_ws=central_base_model_path, verbose=False, check=False, exe_name="mf2000")

In [8]:
top = cbm.dis.top.array[extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
top = scale(scaling_factor, top)

botm = cbm.dis.botm.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
botm = scale(scaling_factor, botm)

In [ ]:
dis = mf.ModflowDis(
    ml, 
    nlay=nlay,
    nrow=nrow, 
    ncol=ncol, 
    delr=delr, 
    delc=delc, 
    top=top, 
    botm=botm, 
    nper=nper, 
    steady=False
)

dis.plot()

### Setup Basic-Package

In [ ]:
ibound = 1
strt = cbm.bas6.strt.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
strt = scale(scaling_factor, strt)
bas = mf.ModflowBas(ml, ibound=ibound, strt=strt)
bas.plot()

### Setup General Head Boundaries

lets take the start values here and extract the values for each border and layer

In [ ]:
#  [lay, row, col, stage, cond]
mask_arr = np.zeros((nlay, nrow, ncol))
mask_arr[:, 0, :] = 1
mask_arr[:, -1, :] = 1
mask_arr[:, :, 0] = 1
mask_arr[:, :, -1] = 1

ghb_spd = {0: []}
for layer_id in range(nlay):
    for row_id in range(nrow):
        for col_id in range(ncol):
            if mask_arr[layer_id][row_id][col_id] == 1:
                ghb_spd[0].append([layer_id, row_id, col_id, strt[layer_id][row_id][col_id], 200])

ghb = mf.ModflowGhb(ml, stress_period_data=ghb_spd)
ghb.plot()

### Setup Recharge

In [ ]:
rech = {}
rech[0] = 0.00033
rch = mf.ModflowRch(ml, rech=rech)

rch.plot()

### Setup the well boundary

Add wells here. Load from CSV and calculate layer.

In [ ]:
#welSp = {0: []}
#wel = mf.ModflowWel(ml, stress_period_data=welSp)
#wel.plot()

### Setup Flow-Package

In [ ]:
bcf = cbm.bcf6
#print (bcf)
hdry = bcf.hdry
wetfct = bcf.wetfct
ihdwet = bcf.ihdwet
ipakcb = bcf.ipakcb
iwdflg = bcf.iwdflg
iwetit = bcf.iwetit
intercellt = bcf.intercellt
laycon = bcf.laycon
trpy = bcf.trpy


hy = bcf.hy.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]

sf1 = bcf.sf1.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
sf2 = bcf.sf2.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
tran = bcf.tran.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
vcont = bcf.vcont.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]
wetdry = bcf.wetdry.array[:, extractRowsBetween[0]:extractRowsBetween[1], extractColsBetween[0]:extractColsBetween[1]]

hy = scale(scaling_factor,hy)
sf1 = scale(scaling_factor,sf1 )
sf2 = scale(scaling_factor,sf2)
tran = scale(scaling_factor,tran)
vcont = scale(scaling_factor,vcont)
wetdry = scale(scaling_factor,wetdry)

bcf = mf.ModflowBcf(    ml,
                        hdry = hdry, 
                        wetfct = wetfct,
                        ihdwet = ihdwet,
                        ipakcb = ipakcb ,
                        iwdflg = iwdflg,
                        iwetit = iwetit,
                        hy = hy,
                        intercellt = intercellt,
                        sf1 = sf1,
                        sf2 = sf2,
                        tran = tran,
                        trpy = trpy ,
                        vcont = vcont,
                        wetdry = wetdry,
                        laycon = laycon,
                        #filenames = 'model.cbc'
                   )


### Setup Solver Package and Output-control with defaults

In [ ]:
pcg = mf.ModflowPcg(ml)
oc = mf.ModflowOc(ml)

In [ ]:
ml.write_input()
ml.run_model()

### Read output file

In [ ]:
hds = fu.HeadFile(os.path.join(landelino_model_path, modelname + '.hds'))
times = hds.get_times()

### Plot output file

In [ ]:
hds.plot(totim=times[-1], colorbar=True, masked_values=[1e30])